In [1]:
# 导入模块

import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import joblib
import sqlite3
import pickle
import nltk

%matplotlib inline

import pymongo
import pandas as pd

In [2]:
# 连接并获得CommitMsg数据库和Collection
myclient = pymongo.MongoClient('localhost', 27017)
mydb = myclient["commitsDB"]
mycol = mydb["commitTrainningSample"]


In [3]:
# 获取commitID，commitMsg和commitIntent
data = mycol.find({},{ "_id": 1, "commitID": 1, "commitMsg": 1, "commitIntent": 1 })
data = list(data)
mydf = pd.DataFrame(data)

print(mydf)

In [4]:
# 清洗commitMsg
nltk.download('stopwords')
stemmer = PorterStemmer()
words = stopwords.words("english")
mydf['cleaned'] = mydf['commitMsg'].apply( \
    lambda x: " ".join([stemmer.stem(i) \
    for i in re.sub("[^a-zA-Z]", " ", x).split() \
    if i not in words]).lower())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Chuncen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# 使用本地模型对cleaned commitMsg进行分类
X = mydf['cleaned']
model = joblib.load('rf.model')
Y = model.predict(X)
Y

array([4, 1, 1, 3, 4, 3, 3, 4, 1, 4, 3, 1, 5, 4, 5, 4])

In [7]:
# 写入新collection里commitTrainningSampleIntent
newcol = mydb["commitTrainningSampleIntent"]
for x in data:
    Yvalue = Y[data.index(x)]
    if Yvalue == 1:
        tag = "Bug fix"
    elif Yvalue == 2:
        tag = "No category"
    elif Yvalue == 3:
        tag = "Design improvement"
    elif Yvalue == 4:
        tag = "New feature"
    elif Yvalue == 5:
        tag = "Non-functional requirements"
    x["commitIntent"] = tag
    newcol.insert_one(x)